In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")


In [ ]:
train.head()
#will assume that the information on the name and ticket is useless/noisy ,or explained by other variables(sex,Pclass,ect)
#will assume that the information inside of the strings on the cabin collumn is useless/noisy

In [ ]:
train.info()
#age,cabin and embarked have null values

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train)
#good variable

In [ ]:
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')
#good variable

In [ ]:
train[train["Survived"]==0]["Age"].hist(bins=100)

In [ ]:
train[train["Survived"]==1]["Age"].hist(bins=100)
#good variable,but becomes noisy for people older that 16/18

In [ ]:
sns.countplot(x="Survived",hue="SibSp",data=train)
#not a good regressor/the data is noisy

In [ ]:
train[train["Survived"]==0]["Fare"].hist(bins=30)

In [ ]:
train[train["Survived"]==1]["Fare"].hist(bins=30)
#not a good regressor/the data is noisy

In [ ]:
sns.countplot(x="Survived",hue="Parch",data=train)
#not a good regressor/the data is noisy

In [ ]:
sns.countplot(x="Survived",hue="Embarked",data=train)
#not a good regressor/the data is noisy

In [ ]:
def is_na(x):
    if pd.isnull(x):
        return 1
    else:
        return 0
train["Cabinisnull"]=train["Cabin"].apply(is_na)
#testing if having a cabin number is a good regressor

In [ ]:
sns.countplot(x="Survived",hue="Cabinisnull",data=train)
#good variable

In [ ]:
def under16(x):
    if x<=16:
        return 1
    else:
        return 0
sex = pd.get_dummies(train['Sex'],drop_first=True)
#making dummy variables with sex
train["Ageis16"]=train["Age"].apply(under16)
#making age a categorical variable , as discusted before
train.drop(['Sex','Embarked','Name','Ticket',"Cabin","Age","Fare","SibSp","Parch",'Embarked'],axis=1,inplace=True)
#droping collums that are not usefull
train = pd.concat([train,sex],axis=1)

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1), 
                                                    train['Survived'], test_size=0.30, 
                                                    random_state=101)

In [ ]:
from xgboost import XGBClassifier
my_model=XGBClassifier(n=1000,n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)],
             verbose=True)

In [ ]:
pred=my_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))


In [ ]:
test["Cabinisnull"]=test["Cabin"].apply(is_na)
sex = pd.get_dummies(test['Sex'],drop_first=True)
test["Ageis16"]=test["Age"].apply(under16)
test.drop(['Sex','Embarked','Name','Ticket',"Cabin","Age","Fare","SibSp","Parch"],axis=1,inplace=True)
test = pd.concat([test,sex],axis=1)

In [ ]:
pred=my_model.predict(test)
final=pd.DataFrame()
final["PassengerId"]=test["PassengerId"]
final["Survived"]=pred
final.to_csv("final",index=False)